In [1]:
!pip3 install pandas dash

     |████████████████████████████████| 7.3 MB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 26.5 MB 21.4 MB/s eta 0:00:01 MB 21.4 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 88.5 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3804 sha256=9bc1ec82ca5824dc9ce97dbfd2f7de6efd920d84003a680157e06832e53e9397
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/e4/f3/16724791571a955a46d54650510c98c04ab7d339626aee27cc
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=acc7a96a8a2550d2941bbb479191a424ef8ef2f0967e282d61d3b304ccd61d0d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/73/d8/8d/92f612c03c895f19bcc56a6c54be7bb41aaa698012a5624f60
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3897 sha256=65e7763aedf61adf2041bdaf1a72

Run the following wget command line in the terminal to download dataset as spacex_launch_dash.csv

In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2022-01-26 21:48:33--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2022-01-26 21:48:34 (25.1 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



The following code is used to create the Dashboard.

In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),   
    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    dcc.Dropdown(id='site-dropdown',
                 options=[{'label': 'All Sites', 'value': 'ALL'},
                          {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                          {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                          {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                          {'label': 'VAFB SLC-4E', 'value':  'VAFB SLC-4E'},
                         ],
                         value='ALL', 
                         placeholder='Select a Launch Site here', 
                         searchable=True
                ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(id='payload-slider', 
                    min=0, max=10000, step=1000,
                    value=[min_payload, max_payload]
                    ),
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value')
             )
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        pie_fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Successful launches')
    else:
        fil = spacex_df['Launch Site'] == entered_site
        print('HERE1', entered_site)
        print('HERE2:', fil)
        fil2 = spacex_df[['Launch Site', 'class']]
        fil3 = fil2[fil]
        fil3 = fil3['class'].value_counts()
        print("FHERE:", fil3)
        sub_fil = []
        sub_fil.clear()
        for f in fil3:
            print('AHERE:', f)
            sub_fil.append(f)
        print('SUB_FIL:', sub_fil)
        if entered_site == 'KSC LC-39A':
            cl = [1, 0]
        else:
            cl = [0, 1]
        dat = {'Launch Site': [entered_site, entered_site], 
               'class': cl, 'count': sub_fil}
        filtered_df = pd.DataFrame(dat)
        print('HERE3:', filtered_df)

        pie_fig = px.pie(filtered_df, values='count', 
        names='class', 
        title=entered_site)

    return pie_fig
        # return the outcomes piechart for a selected site
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, slider_range):
    fil = spacex_df['Launch Site'] == entered_site
    filtered_df = spacex_df[fil]
    low, high = slider_range
    mask = (spacex_df['Payload Mass (kg)'] > float(low)) & (spacex_df['Payload Mass (kg)'] < float(high))
    if entered_site == 'ALL':
        scatter_fig = px.scatter(spacex_df[mask], x='Payload Mass (kg)', y='class',
                                 color='Booster Version Category')
    else:
        scatter_fig = px.scatter(filtered_df[mask], x='Payload Mass (kg)', y='class',
                                 color='Booster Version Category')
    
    return scatter_fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
